In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout
import numpy as np
import sys
sys.path.append("src")
from RINEX import RINEX3_to_obsmat

In [ ]:
#创建GS_LSTM模型(伪距用)
input_shape=(10,32)
out_shape=2
model=Sequential()
model.add(LSTM(12,return_sequences=True,activation="relu",input_shape=input_shape))
model.add(LSTM(24,return_sequences=True,activation="relu"))
model.add(LSTM(12,return_sequences=False,activation="relu"))
model.add(Dropout(0.01))
model.add(Dense(out_shape,activation="softmax"))
optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001)
model.compile(optimizer=optimizer,loss="categorical_crossentropy",metrics=['categorical_accuracy'])
model.summary()

In [ ]:
obs_type=["C1C","L1C","D1C","S1C","C2W","L2W","D2W","S2W"]
wuh=RINEX3_to_obsmat("IGS_data/wuh20610.25o",obs_type=obs_type)#空旷场景训练集
wuh_sim=RINEX3_to_obsmat("IGS_data/wuh20610.25o.sim",obs_type=obs_type)#环境误差场景测试集
shao=RINEX3_to_obsmat("IGS_data/wuhn0610.25o",obs_type=obs_type)#空旷场景测试集
lh21=RINEX3_to_obsmat("sim_data/lh210610.25o.sim",obs_type=obs_type)#环境误差场景训练集,此数据请联系作者并承诺用于学术用途获取

In [ ]:
def obsmat2sequnce(o_mat,id=0,split_second=30,sat_num=32):
    data=[]
    for i in range(len(o_mat)):
        epoch=np.zeros(sat_num,dtype=np.float64)
        if(o_mat[i][0]['GPSsec']%split_second==0 and o_mat[i][0]['s_num']>0 and o_mat[i][0]['Epoch_OK']==0):
            for d in o_mat[i][1]:
                try:
                    prn_id=int(d['PRN'][-2:])-1
                except:
                    continue
                epoch[prn_id]=d['OBS'][id]
            data.append(epoch)
            #print(o_mat[i][1][0]['PRN'])
    return np.array(data)
wuh_npydata=obsmat2sequnce(wuh)
sim_npydata=obsmat2sequnce(wuh_sim)
nlos_npydata=obsmat2sequnce(lh21)
shao_npydata=obsmat2sequnce(shao)

In [ ]:
def npy2norm(npydata):
    norm_results=np.zeros(npydata.shape,dtype=np.float64)
    norms=np.zeros((32,2),dtype=np.float64)
    for i in range(32):
        #首先确定最值
        npy_max=0
        npy_min=999999999999
        for j in range(len(npydata)):
            if npydata[j][i]!=0.0:
                if(npy_max<npydata[j][i]):
                    npy_max=npydata[j][i]
                if(npy_min>npydata[j][i]):
                    npy_min=npydata[j][i]
        norms[i][0]=npy_max
        norms[i][1]=npy_min
        #然后开始归一化
        for j in range(len(npydata)):
            if npydata[j][i]!=0.0:
                norm_results[j][i]=(npydata[j][i]-npy_min)/(npy_max-npy_min)
    return norms,norm_results
def norms2seq(norms,timestep=10):
    results=[]
    for i in range(len(norms)-timestep+1):
        result=[]
        for j in range(i,i+timestep):
            result.append(norms[j])
        results.append(result.copy())
    return np.array(results)
_,wuh_x=npy2norm(wuh_npydata)
wuh_x_seq=norms2seq(wuh_x)
_,sim_x=npy2norm(sim_npydata)
sim_x_seq=norms2seq(sim_x)
_,nlos_x=npy2norm(nlos_npydata)
nlos_x_seq=norms2seq(nlos_x)
_,shao_x=npy2norm(shao_npydata)
shao_x_seq=norms2seq(shao_x)

filepath="best.h5"
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath,
    monitor='val_categorical_accuracy', save_weights_only=True,verbose=1,save_best_only=True, period=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train=[]
Y_train=[]
X_test=[]
Y_test=[]
for d in wuh_x_seq:
    X_train.append(d)
    Y_train.append([1,0])
for d in nlos_x_seq:
    X_train.append(d)
    Y_train.append([0,1])
for d in sim_x_seq:
    X_test.append(d)
    Y_test.append([0,1])
for d in shao_x_seq:
    X_test.append(d)
    Y_test.append([1,0])
X_train,_,Y_train,_=train_test_split(X_train,Y_train,test_size=0.3,random_state=33)


X_train=np.array(X_train)
X_test=np.array(X_test)
Y_train=np.array(Y_train)
Y_test=np.array(Y_test)

model.fit(X_train,Y_train,epochs=100,
          validation_data=(X_test,Y_test),
          callbacks=[checkpoint])

In [ ]:
model.evaluate(X_train[:2871],Y_train[:2871])
model.evaluate(X_train[2871:],Y_train[2871:])
model.evaluate(X_test[:2871],Y_test[:2871])
model.evaluate(X_test[2871:],Y_test[2871:])

In [ ]:
#最优模型验证
input_shape=(10,32)
out_shape=2
tmodel=Sequential()
tmodel.add(LSTM(12,return_sequences=True,activation="relu",input_shape=input_shape))
tmodel.add(LSTM(24,return_sequences=True,activation="relu"))
tmodel.add(LSTM(12,return_sequences=False,activation="relu"))
tmodel.add(Dropout(0.01))
tmodel.add(Dense(out_shape,activation="softmax"))
optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001)
tmodel.compile(optimizer=optimizer,loss="categorical_crossentropy",metrics=['categorical_accuracy'])
tmodel.load_weights("best.h5")

tmodel.evaluate(X_train[:2871],Y_train[:2871])
tmodel.evaluate(X_train[2871:],Y_train[2871:])
tmodel.evaluate(X_test[:2871],Y_test[:2871])
tmodel.evaluate(X_test[2871:],Y_test[2871:])

In [ ]:
pre_sim=tmodel.predict(X_test[:2871])
pre_wuhn=tmodel.predict(X_test[2871:])

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['font.family']="Times New Roman"
results=[]
for i in range(len(pre_sim)):
    if(pre_sim[i][0]<pre_sim[i][1]):
        results.append(1)
    else:
        results.append(0)
green=[]
red=[]
for i in range(len(results)):
    if(results[i]==1):
        green.append(i)
    else:
        red.append(i)

plt.figure(dpi=400,facecolor='white',figsize=(10,5))
plt.subplot(2,1,1)
plt.scatter(red,[0 for t in range(len(red))],c='r')
plt.scatter(green,[1 for t in range(len(green))],c='green')
plt.yticks([0,1],["False","True"],fontsize=15)
plt.xticks([])
plt.legend(["WUH2_SIM False recognition epochs","WUH2_SIM True recognition epochs"])

results=[]
for i in range(len(pre_wuhn)):
    if(pre_wuhn[i][0]>pre_wuhn[i][1]):
        results.append(1)
    else:
        results.append(0)
green=[]
red=[]
for i in range(len(results)):
    if(results[i]==1):
        green.append(i)
    else:
        red.append(i)

plt.subplot(2,1,2)
plt.scatter(red,[0 for t in range(len(red))],c='r')
plt.scatter(green,[1 for t in range(len(green))],c='green')
plt.yticks([0,1],["False","True"],fontsize=15)
plt.xlabel("epochs in day",fontsize=15)
plt.legend(["WUHN False recognition epochs","WUHN True recognition epochs"])